## Lab Exercises: Handling Ambiguity, Troubleshooting & Prompt Failures in Prompt Engineering
***

### Setup:

- Use OpenAI API or any language model API accessible in your Colab.
- If API keys are needed, ensure they are safely added as environment variables or input by the user.

In [1]:
from google.colab import userdata
import os

# Set your OpenAI API key securely in Colab Secrets (once)
# userdata.set("OPENAI_API_KEY", "your-api-key-here")

# Retrieve key in your notebook
openai_api_key = userdata.get("OPENAI_API_KEY")
if openai_api_key:
    os.environ["OPENAI_API_KEY"] = openai_api_key
    print("✅ OpenAI API key loaded safely")
else:
    print("❌ OpenAI API key not found. Please set it using Colab Secrets.")

✅ OpenAI API key loaded safely


In [2]:

!pip install --quiet openai -q
# Create client
from openai import OpenAI
client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

In [3]:
# Helper Function to Send Prompts
def generate_response(prompt, model="gpt-4o-mini", temperature=0.7):
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            temperature=temperature
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        return f"Error: {e}"


## Exercise 1: Identifying Ambiguity in Prompts
### Objective: See how ambiguous prompts produce inconsistent or unclear responses.

In [4]:
# Task:
# Use an ambiguous prompt like:
ambiguous_prompt = "Describe the process."


In [5]:
# Create a clarified version with clear intent and constraints:

clarified_prompt = "Describe the photosynthesis process focusing on light absorption and chemical reactions."


In [6]:
# Run both through generate_response() and compare outputs.
print("=== Ambiguous Prompt ===")
print(generate_response(ambiguous_prompt), "\n")

print("=== Clarified Prompt ===")
print(generate_response(clarified_prompt))


=== Ambiguous Prompt ===
Could you please clarify which process you are referring to? There are many processes in various contexts, such as scientific, industrial, educational, or even personal development. Providing more details will help me give a more accurate and relevant description. 

=== Clarified Prompt ===
Photosynthesis is the process by which green plants, algae, and some bacteria convert light energy into chemical energy stored in glucose. It primarily takes place in the chloroplasts of plant cells and involves two main stages: the light-dependent reactions and the light-independent reactions (Calvin cycle).

### 1. Light Absorption

**Chlorophyll and Accessory Pigments:**
- The primary pigment involved in photosynthesis is chlorophyll, which is found in the thylakoid membranes of chloroplasts. Chlorophyll absorbs light most efficiently in the blue (around 430 nm) and red (around 660 nm) wavelengths, while reflecting green light (around 500-550 nm), which is why plants appe

## Exercise 2: Troubleshooting Inconsistent or Irrelevant Output
### Objective: Practice troubleshooting a vague or failed prompt by refining it step-by-step.

In [7]:
# Task:

# Start with a vague prompt that tends to fail:
vague = "Write a poem."


In [8]:
# Observe shortcomings in the output (such as randomness or lack of clear structure).

# Refine the prompt to add instructions:
refined = "Write a 4-line rhyming poem about spring with a hopeful tone."


In [9]:
# If the model produces unexpected content, add explicit "Important:" instructions to guide it further.
print("=== Vague Prompt ===")
print(generate_response(vague), "\n")

print("=== Refined Prompt ===")
print(generate_response(refined), "\n")

important_refined = """
Important: The poem must not mention winter or cold, focus only on spring themes in 4 lines with rhyme.
Write the poem:
"""
print(generate_response(important_refined))


=== Vague Prompt ===
In a garden where the wildflowers sway,  
Beneath the sun's soft, golden ray,  
Whispers of the breeze weave tales untold,  
Of secrets kept in petals, bright and bold.  

The daisies dance in a playful line,  
While violets blush, their colors divine,  
A symphony of scents fills the air,  
As nature’s brush paints everywhere.  

The trees stand tall, guardians of time,  
Their leaves a chorus, a rustling rhyme,  
Each branch a story, each trunk a song,  
In the heart of this haven, where we belong.  

The sky, a canvas of azure and dreams,  
Reflects in the pond where the sunlight gleams,  
Ripples of laughter, a fleeting embrace,  
In this tranquil sanctuary, we find our place.  

So let us wander where the wildflowers grow,  
With every step, let our spirits glow,  
For in this moment, so simple, so sweet,  
Life’s greatest treasures lie at our feet. 

=== Refined Prompt ===
In springtime's glow, the blossoms bloom,  
Awakening earth from winter's gloom.  
With

## Exercise 3: Diagnosing and Fixing Hallucinations in AI Responses
### Objective: Learn how to manage prompts to reduce AI "hallucinations" (made-up facts).

In [10]:
# Ask the model a fact-based question without anchoring:
hallucination_prompt = "Tell me about the latest Mars mission."


In [11]:
# Then anchor the prompt with fact-stating instructions:

anchored_prompt = """
Based only on verified scientific information up to 2023,
summarize the objectives of the latest Mars mission.
If not sure, say 'I don't know.'
"""


In [12]:
# Compare outputs.

print("=== Potential Hallucination Prompt ===")
print(generate_response(hallucination_prompt), "\n")

print("=== Anchored Prompt to Avoid Hallucinations ===")
print(generate_response(anchored_prompt))


=== Potential Hallucination Prompt ===
As of my last update in October 2023, the most recent Mars mission is NASA's Perseverance rover, which landed on Mars on February 18, 2021. Perseverance is part of NASA's Mars 2020 mission and is designed to explore the Jezero Crater, which is believed to have once contained a lake and might hold signs of ancient microbial life.

Perseverance's primary goals include:

1. **Astrobiology**: Searching for signs of past life and collecting rock and soil samples for potential return to Earth.
2. **Sample Collection**: Collecting and caching samples of Martian rock and regolith for future missions to bring back to Earth.
3. **Testing New Technologies**: Demonstrating new technologies that could be used in future Mars explorations, including the MOXIE experiment that produces oxygen from the Martian carbon dioxide atmosphere.
4. **Understanding Mars' Geology and Climate**: Studying the planet's geology and climate to understand its past and prepare for f

## Exercise 4: Handling Overly Long or Complex Prompts (Troubleshooting Failure Modes)
### Objective: See how very long or overloaded prompts can cause failure or truncated responses.

#### Task:

- Create an overstuffed prompt combining multiple tasks or large amounts of data.

- Send it and observe if the output cuts off or gets confused.

- Then split the task into smaller prompts with focused instructions.

- Compare the results and check for better coherence and completeness.

In [13]:
overloaded_prompt = """
Explain the theory of relativity including special and general relativity,
provide examples, historical context, significance in physics, and common misconceptions,
all in a single paragraph.
""" * 3  # Repeated to simulate length

print("=== Overloaded Prompt ===")
print(generate_response(overloaded_prompt), "\n")

smaller_prompts = [
    "Explain the theory of special relativity in simple terms.",
    "Explain the theory of general relativity with a real-world example.",
    "List common misconceptions about relativity."
]

for i, sp in enumerate(smaller_prompts, 1):
    print(f"--- Smaller Prompt {i} ---")
    print(generate_response(sp), "\n")


=== Overloaded Prompt ===
The theory of relativity, developed by Albert Einstein in the early 20th century, comprises two key components: special relativity (1905) and general relativity (1915). Special relativity introduces the idea that the laws of physics are the same for all observers in uniform motion and reveals that the speed of light is constant regardless of the observer's motion, leading to the famous equation \(E=mc^2\), which illustrates the equivalence of mass and energy; a practical example is the functioning of GPS satellites, which must account for relativistic effects to provide accurate positioning. General relativity expands this framework to include acceleration and gravity, proposing that massive objects like planets warp spacetime, causing other objects to follow curved paths—an example being the way light bends around the sun, confirmed during a solar eclipse in 1919. Historically, these theories challenged the Newtonian concepts of absolute time and space, profo

## Exercise 5: Meta-Prompting for Troubleshooting
### Objective: Use meta-prompts to ask the model to clarify its understanding or identify issues with ambiguous instructions.

In [14]:
# Start with an ambiguous prompt:
ambiguous_request = "Create an efficient sorting algorithm."


In [15]:
# Ask the model if it understands what is meant by "efficient sorting algorithm":

meta_prompt = """
Before you write code, please explain what you understand by
'efficient sorting algorithm.' If unsure, ask for clarification.
"""


In [16]:
# Run meta-prompt and analyze the output.
# Based on the model's answer, refine the original prompt with explicit constraints.

print("=== Ambiguous Request ===")
print(generate_response(ambiguous_request), "\n")

print("=== Meta Prompt (Check Understanding) ===")
print(generate_response(meta_prompt), "\n")

refined_sort_prompt = """
Write a Python function implementing merge sort, emphasizing O(n log n) time complexity.
Include comments explaining each step.
"""
print(generate_response(refined_sort_prompt))



=== Ambiguous Request ===
Creating an efficient sorting algorithm involves understanding the requirements and constraints of the problem at hand. Below, I'll describe the Quick Sort algorithm, which is often considered one of the most efficient sorting algorithms for average cases due to its O(n log n) average time complexity. 

### Quick Sort Algorithm

Quick Sort is a divide-and-conquer algorithm. Here's how it works:

1. **Choose a Pivot**: Select an element from the array as the pivot. Various strategies exist for choosing the pivot, including picking the first element, the last element, a random element, or the median.

2. **Partitioning**: Rearrange the array so that elements less than the pivot come before it and elements greater than the pivot come after it. The pivot is then in its final position.

3. **Recursively Apply**: Recursively apply the above steps to the sub-arrays of elements with smaller values and elements with greater values.

### Quick Sort Implementation in Pyt

#### What Students Will Learn
- How ambiguous or incomplete prompts lead to weak or inconsistent AI outputs.

- Iterative troubleshooting by refining prompts with specificity, structure, and constraints.

- Techniques to avoid hallucinations via anchoring and explicit instructions.

- The importance of task decomposition for complex queries.

- Using meta-prompting to clarify intent and prompt understanding.

- Recognizing failure modes such as length truncation and ambiguous phrasing.